In [1]:
#インポート(ないならpip installでインストール)
import fitz          #PDF読み込むライブラリ(pip install PyMuPDF)
import pandas as pd  #表を生成するライブラリ(pip install pandas)
import numpy as np   #配列を便利に使えるようになるライブラリ(pip install numpy)
import os
import glob
from pathlib import Path
import re

In [2]:
#関数の戻り値を入れる用のデータフレーム
df = pd.DataFrame()

In [3]:
#全角数字を半角数字にする用のテーブル
zenkaku_table = str.maketrans({
    '１': '1',
    '２': '2',
    '３': '3',
    '４': '4',
    '５': '5',
    '６': '6',
    '７': '7',
    '８': '8',
    '９': '9',
    '０': '0',
})

In [15]:
#売上高を抽出する関数
def get_sale(PDF_name):
    try:
        doc = fitz.open(PDF_name) 
    
        #売上高があるページの取得
        array = np.array(doc.get_toc()) 
        page_sales = array[np.any((array == "１主要な経営指標等の推移") | (array == "1主要な経営指標等の推移"), axis = 1), :]
        page_int = int(page_sales[0,2])
        page = doc[page_int - 1]
        
        #会社名の取得
        text = doc.get_page_text(0)
        lines = text.splitlines()
        next_lines = []
        target_string = "会社名"
        for i in range(len(lines) - 1):
            if target_string in lines[i]:
                next_lines.append(lines[i + 1].strip())
                company_name = next_lines[0]
    
        #２ページ目にある売上高を含めた表の抽出
        tables = page.find_tables()
    
        #抽出した表から売上高を抽出する
        table_data = np.array(tables[0].extract())
        columns = table_data[0]
        row_sales = table_data[np.any(table_data == "決算年月", axis = 1),:]
        row_1_sales = np.ravel(row_sales)
    
        #全角を半角にする
        for i in range(row_1_sales.size):
            word = row_1_sales[i]
            row_1_sales[i] = row_1_sales[i].translate(zenkaku_table)
            numbers = re.findall(r'\d+', row_1_sales[i])
        
        #和暦西暦変換
        characters = "昭平令"
        pattern = f"[{re.escape(characters)}]"
        matches = re.findall(pattern, row_1_sales[i])
        if len(numbers) != 0:
                year = numbers[0]
                date = numbers[1]
                if len(matches) != 0:
                    if matches[0] == "令":
                        year = to_seireki('令', year)
                    elif matches[0] == "平":
                        year = to_seireki('平', year)
                    elif matches[0] == "昭":
                        year = to_seireki('昭', year)
                    else:
                        print("違う")
                row_1_sales[i] = year + " - " + date
    
        data_rows = table_data[1:]
        
        #エラーその１.データの形状と列名が不一致(未対策)
        if len(data_rows) > 0 and len(data_rows[0]) != len(row_1_sales):
            raise ValueError("データの列数と列名の数が一致しません")
        
        df = pd.DataFrame(data_rows, columns = row_1_sales)
        sales = df[df['決算年月'].str.contains('売上' , na = False)] 
        
        #エラーとかで売り上げデータが引っ張れなかった場合
        if not sales.empty:
            sales.iat[0, 0] = company_name
        else:
            print(f"{company_name}: 売上データなし")
            
        return sales
    except Exception as e:
        print(f"{PDF_name} のエラー: {e}")
        
        #空のDataFrameを返す
        return pd.DataFrame() 

In [5]:
#売上高だけのデータフレームを作成する関数
def make_dataframe(sales):
    a = pd.DataFrame()
    df = a.append(sales, ignore_index = True)
    return df

In [6]:
#ファイルのパスを取得
def make_file_path_list(dir):
    file_path_list = glob.glob(f'./{dir}/*')
    file_path_list.sort()
    
    return file_path_list

In [12]:
def complate(dir):
    df = pd.DataFrame()
    file_path_list = make_file_path_list(dir)
    for i in range(1,59):
        pdf_name = file_path_list[i]
        sales = get_sale(pdf_name)
        df = pd.concat([df,sales])
    return df

In [16]:
df = complate("yuho_report")

株式会社Ｍマート: 売上データなし
東宝株式会社: 売上データなし
株式会社ビザスク: 売上データなし
沖縄セルラー電話株式会社: 売上データなし
./yuho_report/2023-06-16_E22460.pdf のエラー: name 'to_seireki' is not defined


In [17]:
df

,決算年月,2019年1月,2020年1月,2021年1月,2022年1月,2023 - 1,2019年2月,2020年2月,2021年2月,2022年2月,...,2021年\n2月,2022年\n2月,2019年3月,2020年3月,2021年3月,2022年3月,2023 - 3,2020年3月期,2021年3月期,2022年3月期
1,株式会社シーイーシー,"49, 810, 774","51, 868, 569","48, 003, 297","45, 220, 567","48, 206, 206",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,株式会社ジャストプランニング,"2, 254, 092","2, 426, 749","2, 103, 153","2, 107, 874","2, 007, 240",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,株式会社ユークス,"3, 878, 166","3, 928, 546","2, 650, 178","3, 632, 485","4, 299, 846",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,株式会社エニグモ,"5, 283, 572","6, 097, 281","7, 077, 484","7, 616, 747","6, 868, 805",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,株式会社ネオジャパン,―,"3, 742, 984","5, 325, 021","5, 920, 092","6, 007, 080",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,株式会社アピリッツ,―,―,―,"4, 795, 709","7, 323, 080",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,株式会社ライトワークス,NaN,"1, 217, 500","1, 757, 789","2, 219, 053","2, 640, 880",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,株式会社テクノロジーズ,NaN,NaN,"785, 227","789, 282","1, 075, 505",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ポールトゥウィンホールディングス株式会社,"23, 763, 960","26, 120, 452","26, 729, 396","34, 252, 376","39, 929, 250",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,株式会社col y,"2, 446, 830","3, 359, 421","6, 331, 634","6, 519, 896","5, 537, 488",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
